# **Análisis de venta**

Análisis exploratorio de una tabla de ventas proporcionada por negocio.


## **Contexto de los datos**

Cada fila de la tabla representa una venta individual.
El objetivo de este análisis es entender el comportamiento de las ventas
por producto, canal y fecha, y detectar posibles problemas de calidad
en los datos antes de analizarlos.


## **Carga de datos** 

Se carga el archivo CSV proporcionado por negocio para comenzar
la inspección inicial de los datos.


In [ ]:
import pandas as pd

df = pd.read_csv("../data/raw/ventas_sucias.csv")

### Inspección inicial

Primera visualización de los datos para entender estructura y contenido general.


In [14]:
df.head()

,id_venta,fecha,cliente,producto,precio,cantidad,canal
0,1,2024-01-05,Ana López,Teclado,25.5,2.0,web
1,2,05/01/2024,Carlos Ruiz,Ratón,15.0,1.0,tienda
2,3,2024-01-06,NaN,Monitor,180.0,1.0,web
3,4,2024-01-06,Marta Pérez,Teclado,25.5,NaN,tienda
4,5,2024/01/07,Juan Gómez,Portátil,850.0,1.0,web


### Estructura y tipos de datos

Se revisa la estructura del DataFrame para identificar tipos de datos,
valores nulos y posibles problemas antes de realizar cualquier limpieza.


In [15]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id_venta  8 non-null      int64  
 1   fecha     8 non-null      object 
 2   cliente   7 non-null      object 
 3   producto  8 non-null      object 
 4   precio    8 non-null      float64
 5   cantidad  7 non-null      float64
 6   canal     7 non-null      object 
dtypes: float64(2), int64(1), object(4)
memory usage: 580.0+ bytes


In [16]:
df.isna().sum()

id_venta    0
fecha       0
cliente     1
producto    0
precio      0
cantidad    1
canal       1
dtype: int64

## Observaciones iniciales

A partir de la inspección inicial se observan los siguientes problemas:

- Existen valores nulos en las columnas `cliente`, `cantidad` y `canal`.
- La columna `fecha` está almacenada como texto y no como datetime.
- La columna `cantidad` debería contener valores enteros positivos.
- Hay datos que no cumplen las reglas mínimas de negocio para una venta válida.

## <u> Objetivos del Análisis </u>

El objetivo de este análisis es obtener una visión general del rendimiento
de las ventas a partir de la información disponible de la tabla.

En concreto, se debe responder a las siguientes cuestiones:

- ¿Cuántas ventas se han realizado en total?
- ¿Cuáles son los clientes que más y menos han comprado?
- ¿Qué productos se venden con mayor y menor frecuencia?
- ¿Cuánto dinero se ha gastado cada cliente?
- ¿Cuál es el gasto medio por cliente?
- ¿Cómo evolucionan las ventas a lo largo del tiempo?
- ¿Qué día se vendió más y qué día se vendió menos?
- ¿Qué canal de venta genera mayores ingresos?


## Reglas de negocio para el análisis

Para que una venta sea considerada válida en este análisis, debe cumplir las siguientes condiciones:

- Toda venta debe tener un cliente asociado.
- Cantidad vendida debe existir y ser mayor que cero.
- Precio debe ser un valor numérico válido.
- La fecha debe poder interpretarse correctamente como una fecha.
- El canal de venta debe estar informado o clasificarse como desconocido.


## Preparación para la limpieza

Copia del DataFrame original para poder comparar el estadob de los datos antes 
y después del proceso de limpieza.


In [17]:
df_raw = df.copy()

## Limpieza de datos

A continuación se realizan las transformaciones necesarias para garantizar
la calidad de los datos antes de realizar el análisis.


### Normalización de la columna fecha

Se convierte la columna `fecha` a tipo datetime para permitir análisis temporal.


In [18]:
df["fecha"] = pd.to_datetime(df["fecha"], errors="coerce")
df["fecha"].isna().sum()


np.int64(2)

Tras la conversión de la columna `fecha` a datetime se detectan
dos registros con fechas no válidas, que deberán tratarse
según las reglas de negocio definidas.


In [19]:
df[df["fecha"].isna()]

,id_venta,fecha,cliente,producto,precio,cantidad,canal
1,2,NaT,Carlos Ruiz,Ratón,15.0,1.0,tienda
4,5,NaT,Juan Gómez,Portátil,850.0,1.0,web


In [20]:
df_raw.loc[[1, 4], ["id_venta", "fecha", "cliente", "producto", "precio", "cantidad", "canal"]]

,id_venta,fecha,cliente,producto,precio,cantidad,canal
1,2,05/01/2024,Carlos Ruiz,Ratón,15.0,1.0,tienda
4,5,2024/01/07,Juan Gómez,Portátil,850.0,1.0,web


In [21]:
df_raw.loc[[1, 4], "fecha"].tolist()

['05/01/2024', '2024/01/07']

In [24]:
pd.to_datetime(df_raw["fecha"], errors="coerce", dayfirst=True)


0   2024-05-01
1          NaT
2   2024-06-01
3   2024-06-01
4          NaT
5   2024-07-01
6   2024-08-01
7   2024-08-01
Name: fecha, dtype: datetime64[ns]

In [25]:
pd.to_datetime(df_raw["fecha"], errors="coerce", dayfirst=True).isna().sum()


np.int64(2)

In [26]:
df["fecha"] = pd.to_datetime(
    df_raw["fecha"],
    errors="coerce",
    dayfirst=True
)

In [27]:
df["fecha"].isna().sum()

np.int64(2)